In [ ]:
import sqlite3, os, requests, json,sys, hashlib

In [ ]:
import pandas as pd

In [ ]:
import mutagen, tempfile, hashlib

In [ ]:
from io import BytesIO

In [ ]:
from PIL import Image

In [ ]:
from mutagen.mp3 import MP3
from mutagen.id3 import ID3, APIC

In [ ]:
#填入自己的webdb路径，一般都在下面这个位置（换成自己用户名）
web_library=("C:\\Users\\90500\AppData\\Local\\Netease\\CloudMusic\\Library\\webdb.dat")
#填入网易云api服务器路径，见：https://binaryify.github.io/NeteaseCloudMusicApi/#/
#我不保证自己搭的API的可用性！建议自己npx一个服务器
cloudmusic_api = "http://ncm.lzfluorite.xyz"
#填入网易云歌曲下载目录
cloudmusic_root= "F:\\CloudMusic"

In [ ]:
# collect radio basic info from NCM sqlite
conn = sqlite3.connect(web_library)
("database ready")
sql_query=pd.read_sql_query(
    '''SELECT track_name, artist_name, relative_path, type_extra 
    FROM web_offline_track 
    WHERE type_extra IS NOT NULL
    ''',conn)
conn.close()
("Database connection closed")
df = pd.DataFrame(sql_query)

In [ ]:
# using request to save retrieve file
def retrieveWebImg(img_url):
    response = requests.get(img_url)
    if response.status_code !=200:
        return None
    image = Image.open(BytesIO(response.content))
    return image

In [ ]:
def retrieveEmbededImg(audio_path_str):
    audio_path = os.path.join(audio_path_str)
    tags = ID3(audio_path)
    if tags.get("APIC") is None:
        #貌似有一些明明有但是读不到的现象，顶多覆盖一次，不影响使用
        return None
    pict = tags.get("APIC:").data
    return Image.open(BytesIO(pict))

In [ ]:
def compareImage(image1, image2):
    return hashlib.md5(image1.tobytes()).hexdigest()==hashlib.md5(image2.tobytes()).hexdigest()

In [ ]:
def embedArt(audio_path, image_struct):
    #assuming they are all mp3, haven't seen other forms in radio so far.
    audio = MP3(audio_path, ID3=ID3)
    #delete previous album art
    audio.tags.delall("APIC")
    #create a temporary file to store online cover
    fd, path = tempfile.mkstemp(suffix="."+image_struct.format)
    try:
        image_struct.save(path)
        with open(path, 'rb') as albumart: 
            audio.tags.add(APIC(mime=Image.MIME[image_struct.format],type=3,desc=u'Cover',data=albumart.read()))
            audio.tags.save(audio_path)
    finally:
        os.close(fd)
        os.remove(path)

In [ ]:
missing, updated, total = 0, 0, 0
for track, path, id in zip(df["track_name"],df["relative_path"],df["type_extra"]):
    total += 1
    str_id = str(id)
    para = {"id":str_id}
    print("tracking:"+track+"||"+str_id)
    #retrieve cover art url from NCM api
    r = requests.get(cloudmusic_api +"/dj/program/detail",params=para)
    if r.status_code != 200:
        # radio deleted? skip.
        print(r.status_code)
        print(r.text)
        missing +=1
        continue
    cover_url = json.loads(r.text)["program"]["coverUrl"]
    # print("debug:"+cover_url)
    #get the audio path
    absolute_audio_path = os.path.join(cloudmusic_root,path)
    #get online cover
    web_img = retrieveWebImg(cover_url)
    #get embedded cover
    embeded_img = retrieveEmbededImg(absolute_audio_path)
    if embeded_img is None:
        print("no embedded img")
        temp_bool = False
    else:
        temp_bool = compareImage(web_img,embeded_img)
    #only update cover art if web art differs from embedded art || no embedded art
    if not temp_bool:
        embedArt(absolute_audio_path,web_img)
        print("updated:"+track)
        updated += 1
    else:
        print("same albumart md5, skipped")
print("Task completed! Total:{0},Updated:{1},Missing:{2}".format(str(total),str(updated),str(missing)))